In [10]:
from itertools import islice
import pandas as pd
import numpy as np
from scipy import stats
from  qgrid import show_grid as show 
import plotly.express as px
import math
import seaborn as sns
from matplotlib import pyplot as plt 
import matplotlib as mpl
from matplotlib.lines import Line2D 
# pd.options.plotting.backend = "plotly"
pd.options.plotting.backend ='matplotlib'

mpl.rcParams['boxplot.boxprops.color'] ='white'
mpl.rcParams['boxplot.boxprops.linewidth']=2
mpl.rcParams['patch.edgecolor']='k'
mpl.rcParams['boxplot.patchartist']= "True"
mpl.rcParams['boxplot.flierprops.marker'] = 'o'
mpl.rcParams['boxplot.flierprops.color'] = 'k'
mpl.rcParams['boxplot.flierprops.markerfacecolor'] = "white"
mpl.rc('xtick', labelsize=28) 
mpl.rc('ytick', labelsize=28) 
mpl.rcParams['text.usetex'] = True


In [11]:
def read_top(filename):
    f= open(filename,'r')
    line = f.readline()
    ts=0
    filename=filename.split("/")[1]
    df=pd.DataFrame(columns=["ts","cmd","time","cpu","timestamp"])
    jvm=filename.split("_")[1]
    benchmark=filename.split("_")[0]
    
    while line != '' :
        if line[:3] == "top":
            
            timestamp = line.split(" ")[2]
            f.readline() # 
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            thread = f.readline().split()
            while len(thread) >0 : 
                x = {"ts":ts,
                     "jvm":jvm,
                     "benchmark":benchmark,
                     "status":thread[7],
                    "cmd":" ".join(thread[11:]),
                    "time":thread[10],
                    "cpu":thread[8],
                    "timestamp":timestamp
                }
                df=df.append(x,ignore_index=True)
                thread = f.readline().split()
            ts+=0.5
        line = f.readline()
        df["type"]=df["cmd"].apply(lambda row: "node" if row[:4]=="node" else "others")
    return df 

# df =read_top(filename="records/avrora_small_15.0.1-open")

In [36]:

def plot_threads(df,benchmark,save =False ):
    title=f"Number of active Threads in different Jvms over time when executing {benchmark} "
    y=df.loc[df["benchmark"]==benchmark]
    y=y.loc[y["ts"]>1]
    y=y.sort_values("ts")
    y=y.loc[y["status"]=="R"].groupby(["ts","jvm"]).count().reset_index()
    x=y.pivot(index=["ts"],columns=["jvm"],values="cpu").reset_index().fillna(0)

    
    x=x.sort_values("ts")
    ts=x.iloc[len(x)-1]["ts"]+1
    x=x.append({"ts":ts}, ignore_index=True)
    fig, axes = plt.subplots(1, 1, figsize=(32, 12))

    ###### average mean 
    meanj9=y.loc[y["jvm"]=="15.0.1.j9-adpt"]
    meanj9=meanj9["cpu"].sum()/meanj9.iloc[-1]["ts"]/2
    
    meangraal=y.loc[y["jvm"]=="20.2.0.r11-grl"]
    meangraal=meangraal["cpu"].sum()/meangraal.iloc[-1]["ts"]/2
    
    meanopen15=y.loc[y["jvm"]=="15.0.1-open"]
#      return meanopen15
    meanopen15=meanopen15["cpu"].sum()/meanopen15.iloc[-1]["ts"]/2
   
    meanopen8=y.loc[y["jvm"]=="8.0.265-open"]
    meanopen8=meanopen8["cpu"].sum()/meanopen8.iloc[-1]["ts"]/2
    
    
    ######### legends 
    
    
    axes.plot(x["ts"],x["20.2.0.r11-grl"],'-.',linewidth=3,color="#16A085",label=f'GraalVM: {meangraal:1.1f} Thread/s')
    axes.plot(x["ts"],x["15.0.1.j9-adpt"],'--',linewidth=3,color="#3498DB",label=f'J9: {meanj9:1.1f} Thread/s')
    axes.plot(x["ts"],x["8.0.265-open"],':',linewidth=3,color="#ea5f5b",label=f'HotSpot-8: {meanopen8:1.1f} Thread/s')
    axes.plot(x["ts"],x["15.0.1-open"],'-',linewidth=3,color="#F0B27A",label=f'HotSpot-15: {meanopen15:1.1f} Thread/s')
    
    axes.legend(fontsize=25,loc='center right')
    axes.set_ylim(bottom=0)
    axes.set_ylabel(ylabel="Active Software Threads Count",labelpad=22,fontsize="30")
    axes.set_xlabel(xlabel="Time (s)",labelpad=22,fontsize="30")
    plt.tight_layout()
    if save :
        axes.set_title("",fontsize="30",pad=50  )
        plt.savefig(f"results_threads/sactive_threads_{benchmark}.pdf",format="pdf")
    else:
        axes.set_title(title,fontsize="40",pad=50  )
        plt.show()
    return fig



In [38]:
# l = !(find new_htop -name "*.top")
# data=[read_top(x) for x in l ]
# data=pd.concat(data)
# data=pd.read_csv("all_records.csv")

In [39]:
data

,ts,cmd,time,cpu,timestamp,benchmark,jvm,status,type
0,0,java,0:00.13,86.7,13:54:27,avrora,15.0.1-open,R,others
1,0,C1 Compil+,0:00.04,26.7,13:54:27,avrora,15.0.1-open,R,others
2,0,C1 Compil+,0:00.04,26.7,13:54:27,avrora,15.0.1-open,R,others
3,0,C2 Compil+,0:00.02,13.3,13:54:27,avrora,15.0.1-open,S,others
4,0,java,0:00.00,0.0,13:54:27,avrora,15.0.1-open,S,others
...,...,...,...,...,...,...,...,...,...
2191,13,ForkJoinP+,0:01.84,0.0,14:11:35,scrabble,8.0.265-open,S,others
2192,13,ForkJoinP+,0:01.84,0.0,14:11:35,scrabble,8.0.265-open,S,others
2193,13,ForkJoinP+,0:01.85,0.0,14:11:35,scrabble,8.0.265-open,S,others
2194,13,ForkJoinP+,0:01.80,0.0,14:11:35,scrabble,8.0.265-open,S,others


In [37]:
[plot_threads(x,i,True)for i in data.benchmark.unique()]

KeyError: 'benchmark'